# Chapter 19: Discrete Optimization

In [23]:
import numpy as np
from dataclasses import dataclass
from multipledispatch import dispatch
from itertools import combinations

## Algorithm 19.1

In [25]:
@dataclass
class MixedIntegerProgram:
    A: np.ndarray
    b: list
    c: list
    D: np.ndarray

## Algorithm 19.2

In [6]:
relax = lambda MIP: LinearProgram(MIP.A, MIP.b, MIP.c) # from algorithm 11.1
def round_ip(MIP):
    x = minimize_lp_init(relax(MIP)) # from algorithm 11.5
    for i in MIP.D:
        x[i] = int(x[i])
    return x

## Algorithm 19.3

In [63]:
isint = lambda x, epsilon=1e-10 : abs(round(x)-x) <= epsilon
@dispatch(np.ndarray)
def is_totally_unimodular(A):
    if np.any([a not in [0,-1,1] for a in A.reshape(-1,)]):
        return False
    r, c = A.shape
    for i in range(min([r,c])):
        for a in list(combinations(np.arange(0,r,1),i+1)):
            for b in list(combinations(np.arange(0,c),i+1)):
                B = A[np.ix_(list(a), list(b))]
                if np.linalg.det(B) not in [0,-1,1]:
                    return False
    return True

@dispatch(MixedIntegerProgram)
def is_totally_unimodular(MIP):
    return is_totally_unimodular(MIP.A) and np.all(isint(MIP.b)) and np.all(isint(MIP.c))

### Example

In [64]:
A_1 = np.array([
    [1, 0, 1],
    [0,0,0],
    [1, 0, -1]
])

print(f"1st Matrix is unimodular? -> {is_totally_unimodular(A_1)}")

A_2 = np.array([
    [1, 0, 1],
    [0,0,0],
    [1, 0, 0]
])

print(f"2nd Matrix is unimodular? -> {is_totally_unimodular(A_2)}")

A_3 = np.array([
    [-1, -1, 0, 0, 0],
    [1, 0, -1, -1, 0],
    [0, 1, 1, 0, -1]
])

print(f"3rd Matrix is unimodular? -> {is_totally_unimodular(A_3)}")


1st Matrix is unimodular? -> False
2nd Matrix is unimodular? -> True
3rd Matrix is unimodular? -> True
